In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11763366750982099099,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6529366361
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 7660855360603690806
 physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [4]:
import os
original_dataset_dir = "./datasets/"
base_dir = "./datasets/foodimage"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
validation_dir = os.path.join(base_dir, "validation")

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 증식 설정
train_dataGen = ImageDataGenerator(rescale=1./255,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

test_dataGen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_dataGen.flow_from_directory(train_dir,
                                                   target_size=(224, 224),
                                                   batch_size=32,
                                                   class_mode="categorical")
test_generator = test_dataGen.flow_from_directory(test_dir,
                                                 target_size=(224, 224),
                                                 batch_size=32,
                                                 class_mode="categorical")

Found 36000 images belonging to 10 classes.
Found 12000 images belonging to 10 classes.


In [10]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, History
from tensorflow.keras import backend as K

model = Sequential()
model.add(Conv2D(32, 7, 7, input_shape=(3, 244, 244), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, 5, 5), activation="relu")
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, 3, 3), activation="relu")
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128), activation="relu")
model.add(Dropout(0.5))
model.add(Dense(10), activation="softmax")
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #initial lr = 0.01
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.summary()

ValueError: Negative dimension size caused by subtracting 7 from 3 for '{{node conv2d_2/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 7, 7, 1], use_cudnn_on_gpu=true](Placeholder, conv2d_2/Conv2D/ReadVariableOp)' with input shapes: [?,3,244,244], [7,7,244,32].